In [20]:
import numpy as np
class Variable():
    __counter = 0
    def __init__(self,data,is_leaf=True,backward_fun=None,requires_grad=True):
        if backward_fun is None and not is_leaf:
            raise ValueError('non leaf nodes require backward_fun')
        if np.isscalar(data):
            data = np.ones(1)*data
        if not isinstance(data,np.ndarray):
            raise ValueError(f'data should be of type "numpy.ndarray" or a scalar,but received {type(data)}')
        self.data = data
        self.requires_grad = requires_grad
        if self.requires_grad:
            self.id = Variable.__counter
            print(self.id)
            Variable.__counter += 1
            self.is_leaf = is_leaf
            self.prev = []
            self.backward_fun = backward_fun
            self.grad = Variable(np.zeros(data.shape),requires_grad=False)
        
    def backward(self):
        self.backward_fun(dy=self.grad)
        
    def __repr__(self):
        if self.requires_grad:
            return f'Variable(id:{self.id}, data:{self.data}, grad:{self.grad.data}, prev:{list(map(lambda a:a.id,self.prev))}, is_leaf:{self.is_leaf})\n'
        else:
            return f'Variable(data:{self.data})'

def transpose(a):
    if not (isinstance(a,Variable)):
        raise ValueError('a needs to be a Variable instance')
    def b_fun(dy):
        if a.requires_grad:
            a.grad = plus(grad,transpose(dy))
    res = Variable(a.data.T,is_leaf=False,backward_fun=b_fun)
    res.prev.append(a)
    return res
        
def plus(a,b):
    if not (isinstance(a,Variable) and isinstance(b,Variable)):
        raise ValueError('a,b needs to be a Variable instance')
    def b_fun(dy):
#         import pdb;pdb.set_trace()
        if a.requires_grad:
            new_grad_a = plus(a.grad,plus(b.grad,dy))
        if b.requires_grad:
            b.grad = plus(b.grad,plus(a.grad,dy))
        if a.requires_grad:
            a.grad = new_grad_a
    res = Variable(a.data+b.data,is_leaf=False,backward_fun=b_fun)
    res.prev.extend([a,b])
    return res

def matmul(a,b):
    if not (isinstance(a,Variable) and isinstance(b,Variable)):
            raise ValueError('a,b needs to be a Variable instance')
    def b_fun(dy):
        if a.requires_grad:
            a.grad = plus(a.grad,matmul(dy,transpose(b)))
        if b.requires_grad:
            b.grad = plus(b.grad,matmul(transpose(a),dy))
    res = Variable(np.matmul(a.data,b.data),is_leaf=False,backward_fun=b_fun)
    res.prev.extend([a,b])
    return res

def c_mul(a,c):
    if not (isinstance(a,Variable) and isinstance(c,(int, float))):
        raise ValueError('a needs to be a Variable, c needs to be one of (int, float)')
    def b_fun(dy=1):
        if a.requires_grad:
            a.grad = plus(a.grad,c_mul(dy,c)) 
    res = Variable(a.data*c,is_leaf=False,backward_fun=b_fun)
    res.prev.append(a)
    return res

def top_sort(var):
    vars_seen = set()
    top_sort = []
    def top_sort_helper(vr):
        if (vr in vars_seen) or vr.is_leaf:
            pass
        else:
            vars_seen.add(vr)
            for pvar in vr.prev:
                top_sort_helper(pvar)
            top_sort.append(vr)    
    top_sort_helper(var)
    return top_sort

def backward_graph(var):
    if not isinstance(var,Variable):
        raise ValueError('var needs to be a Variable instance')
    tsorted = top_sort(var)
    
    var.grad=Variable(np.ones(var.data.shape),requires_grad=False)
    for var in reversed(tsorted):
        var.backward()

In [21]:
l1 = Variable(2)
l2 = Variable(3)
r = plus(l1,l2)
print(l1,l2,r)
backward_graph(r)
print(l1,l2,r)

0
1
2
Variable(id:0, data:[2.], grad:[0.], prev:[], is_leaf:True)
 Variable(id:1, data:[3.], grad:[0.], prev:[], is_leaf:True)
 Variable(id:2, data:[5.], grad:[0.], prev:[0, 1], is_leaf:False)

y
3
4
5
6
Variable(id:0, data:[2.], grad:[1.], prev:[], is_leaf:True)
 Variable(id:1, data:[3.], grad:[1.], prev:[], is_leaf:True)
 Variable(id:2, data:[5.], grad:[1.], prev:[0, 1], is_leaf:False)



In [12]:
l1 = Variable(2)
l2 = Variable(3)

n1 = c_mul(l1,2)
n2 = plus(n1,l2)
n3 = matmul(n2,n2)
# print(c,d)
# t_sort = top_sort(d)
# print(t_sort)
print(l1,l2,n1,n2,n3)
backward_graph(n3)
print(l1,l2,n1,n2,n3)

<built-in function id>
<built-in function id>
<built-in function id>
<built-in function id>
<built-in function id>
Variable(id:7, data:[2.], grad:[0.], prev:[], is_leaf:True)
 Variable(id:8, data:[3.], grad:[0.], prev:[], is_leaf:True)
 Variable(id:9, data:[4.], grad:[0.], prev:[7], is_leaf:False)
 Variable(id:10, data:[7.], grad:[0.], prev:[9, 8], is_leaf:False)
 Variable(id:11, data:[49.], grad:[0.], prev:[10, 10], is_leaf:False)

<built-in function id>
<built-in function id>
<built-in function id>
<built-in function id>
<built-in function id>
<built-in function id>
<built-in function id>
<built-in function id>
<built-in function id>
<built-in function id>
<built-in function id>
<built-in function id>
Variable(id:7, data:[2.], grad:[28.], prev:[], is_leaf:True)
 Variable(id:8, data:[3.], grad:[14.], prev:[], is_leaf:True)
 Variable(id:9, data:[4.], grad:[14.], prev:[7], is_leaf:False)
 Variable(id:10, data:[7.], grad:[14.], prev:[9, 8], is_leaf:False)
 Variable(id:11, data:[49.], gra

In [13]:
print(l2.grad)

AttributeError: 'Variable' object has no attribute 'id'